In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import math
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
from itertools import combinations
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression

In [2]:
def Clean_date(df):
    """
    Transfer value in "Collection_Date" column in to Datetime object
    """   
    df["Collection_Date"] = list(map(lambda date: datetime.strptime(date, '%Y-%m-%d %H:%M:%S'),
                                     df["Collection_Date"]))
    return df

In [9]:
def Clean_date(df):
    """
    Transfer value in "Collection_Date" column in to Datetime object
    """   
    df["Collection_Date"] = list(map(lambda date: datetime.strptime(date[:10], '%Y-%m-%d'),
                                     df["Collection_Date"]))
    return df

In [3]:
Station_list = ["POLESOUT", "KISSR0.0", "LZ2", "S133", "TCNS228", "S135",
                "FEBIN", "MBOXSOU", "MH24000", "FEBOUT", "L005", "L008", "LZ40", "L004", "S308C",
                "PALMOUNT", "S169", "S236", "POLE3S", "RITTAE2", "LZ2FA", "L007", "PELBAY3", "L006", "LZ30"]
Variable_list = ['NITRATE+NITRITE-N', 'NITRITE-N', 'AMMONIA-N', 'KJELDAHL NITROGEN, TOTAL', 'PHOSPHATE, ORTHO AS P',
                 'PHOSPHATE, TOTAL AS P', 'NITRATE-N', 'SILICA', 'CARBON, TOTAL ORGANIC', 'CARBON, DISSOLVED ORGANIC',
                 'TOTAL NITROGEN', 'PHOSPHATE, DISSOLVED AS P', 'KJELDAHL NITROGEN, DIS', 'CARBON, TOTAL', 
                 'CARBON, TOTAL INORGANIC', 'NITROGEN, TOTAL DISSOLVED', 'CHLOROPHYLL-A(LC)']

Others = ['CHLOROPHYLL-A', 'PHEOPHYTIN', 'CHLOROPHYLL-A, CORRECTED', 
         'CHLOROPHYLL-C', 'CAROTENOIDS', 'CHLOROPHYLL-B', 'CHLOROPHYLL-A(LC)', 
         'PHEOPHYTIN-A(LC)', 'CHLOROPHYLL-B(LC)', 'RESP. PLANKTONIC']

Nitrogen_list = ['NITRATE+NITRITE-N', 'NITRITE-N', 'AMMONIA-N', 'KJELDAHL NITROGEN, TOTAL',
                'TOTAL NITROGEN','NITRATE-N','KJELDAHL NITROGEN, DIS','NITROGEN, TOTAL DISSOLVED']
Phosphorus_list = ['PHOSPHATE, ORTHO AS P','PHOSPHATE, TOTAL AS P','PHOSPHATE, DISSOLVED AS P']
Carbon_list = ['CARBON, TOTAL ORGANIC', 'CARBON, DISSOLVED ORGANIC','CARBON, TOTAL','CARBON, TOTAL INORGANIC']

In [10]:
def Data_clean_up_3(data, monthly_average=False):
    """
    Similar to Data_clean_up_1, only change the Variable_list 
    and combining the chlorophyll-A (corrected) with chlorophyll-A (LC)
    """

    #clean date: 
    data = Clean_date(data)
    Variable_list = ['CHLOROPHYLL-A(LC)', 'CHLOROPHYLL-A, CORRECTED']
    if monthly_average:
        Time_array = []
        for year in range(2000, 2020):
            for month in range(1,13):
                Time_array.append(datetime(year, month, 1))

        clean_data = pd.DataFrame({"MonthYear": Time_array})

        for variable in Variable_list:
            Value_array = []
            for time in Time_array:
                #select rows based on the given date and variable:
                index_given_time = [True if (x.month == time.month and x.year == time.year) else False for x in data["Collection_Date"]]
                index_given_variable = data["Test Name"] == variable
                #combine 2 criteria:
                index_to_choose = index_given_time & index_given_variable

                values = data.loc[index_to_choose, "Value"]
                #Filter negative and NA values:
                values = [x for x in values if (not math.isnan(x)) and x >= 0]

                if len(values) > 0:
                    Value_array.append(np.mean(values))

                else:
                    Value_array.append(None)

            clean_data[variable] = Value_array
        clean_data.fillna(value=np.nan,inplace=True)
        
    else:
        group_data = []
        for variable in Variable_list:
            #select rows based on the given date and variable:
            index_given_variable = data["Test Name"] == variable
            values = data.loc[index_given_variable, ["Collection_Date", "Value"]]
            #Filter negative and NA values:
            values.dropna(inplace=True)
            values = values.loc[values["Value"]>=0]
            values = values.rename(columns={"Value":variable})
            group_data.append(values)
        #clean_data = pd.concat(group_data)
        clean_data = group_data[0]
        for data in group_data[1:]:
            clean_data = clean_data.merge(data, on=["Collection_Date"], how='left')
        
    if sum(list(clean_data[['CHLOROPHYLL-A(LC)', 'CHLOROPHYLL-A, CORRECTED']].describe().loc["count",:])) != 0:
        clean_data["FINAL CHLOROPHYLL-A"] = np.nanmean(clean_data[['CHLOROPHYLL-A(LC)', 'CHLOROPHYLL-A, CORRECTED']], axis=1)
    return clean_data

In [24]:
station_data.columns

Index(['CHLOROPHYLL-A(LC)', 'Date', 'CHLOROPHYLL-A, CORRECTED', 'Station'], dtype='object')

In [26]:
full_data = []
for station in Station_list:
    station_data = Data_clean_up_3(pd.read_csv(f"{station}.csv"), monthly_average=False).sort_values(by='Collection_Date', ascending=True)
    if "FINAL CHLOROPHYLL-A" in list(station_data.columns):
        station_data.rename(columns={"Collection_Date":"Date"}, inplace=True)
        station_data.rename(columns={"FINAL CHLOROPHYLL-A":"In Situ ChlA"}, inplace=True)
        station_data["Station"] = [station for i in range(len(station_data))]
        full_data.append(station_data[["Date","Station","In Situ ChlA"]])

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (9,14,15,18,25,27,31,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (17,26,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (9,17,18,26,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (18,26,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivi